In [1]:
import numpy as np
import cupy as cp
from scipy.linalg import inv, expm
import networkx as nx
import time
import cupyx.scipy.linalg as cpx_scipy
from cupyx.profiler import benchmark


# Numpy vs CuPy -> Expm

In [9]:
n = 1000
time_val = 5
graph = nx.cycle_graph(n)
adjm = nx.to_numpy_array(graph)

start_time = time.time()
operatorExpm = expm(-1j * adjm * time_val)
end_time = time.time()
numpy_time = end_time - start_time
print(f"NumPy Expm execution time: {numpy_time} seconds")

NumPy Expm execution time: 0.8576884269714355 seconds


In [10]:
n = 1000
time_val = 10
graph = nx.cycle_graph(n)
adjm_cupy = cp.array(nx.to_numpy_array(graph))

start_time = time.time()
operatorCupyExpm = cpx_scipy.expm(-1j * adjm_cupy * time_val)
cp.cuda.Stream().synchronize()
end_time = time.time()
cupy_time = end_time - start_time
print(f"CuPyExpm execution time: {cupy_time} seconds")



CuPyExpm execution time: 0.4162027835845947 seconds


In [12]:
# def calculateExpm_cupy(adjm_cupy, time_val):
#     return cpx_scipy.expm(-1j * adjm_cupy * time_val)

# print("Using CuPy benchmarking tool")

# print(benchmark(calculateExpm_cupy,(adjm_cupy, time_val,), n_repeat=10))

# I dont really understand why the CPU and GPU results from this function are the same, 
#but GPU is half of the time in the cells above.

Using CuPy benchmarking tool
calculateExpm_cupy  :    CPU: 359236.500 us   +/- 5764.629 (min: 350224.300 / max: 370908.700) us     GPU-0: 359290.167 us   +/- 5763.343 (min: 350266.357 / max: 370953.217) us


In [16]:
import numpy as np

# Convert the CuPy array to a NumPy array
operatorCupyExpm_toNp= cp.asnumpy(operatorCupyExpm)

# Compare the two arrays using np.allclose
are_close = np.allclose(operatorExpm, operatorCupyExpm_toNp, atol=1e-5)

# Print the result of the comparison
print(f"Are the two arrays approximately equal? {are_close}")


Are the two arrays approximately equal? False


# Numpy vs CuPy -> Spectral Decomp

## Numpy

In [11]:
n = 5000
time_val = 5
graph = nx.cycle_graph(n)
adjm = nx.to_numpy_array(graph)

start_time = time.time()

eigenvalues, eigenvectors = np.linalg.eigh(adjm)
diag = np.diag(np.exp(-1j * eigenvalues * time_val)).diagonal()
operatorSpectral = np.multiply(eigenvectors, diag)
operatorSpectral = np.matmul(operatorSpectral, eigenvectors.conjugate().transpose())


end_time = time.time()
numpy_time = end_time - start_time
print(f"Numpy execution time: {numpy_time} seconds")

Numpy execution time: 23.47902488708496 seconds


In [27]:
import numpy as np

# Compare the two arrays using np.allclose
are_close = np.allclose(operatorExpm, operatorSpectral, atol=1e-5)

# Print the result of the comparison
print(f"Are the two arrays approximately equal? {are_close}")

Are the two arrays approximately equal? True


## Cupy

In [17]:
n = 10000
time_val = 5
graph = nx.cycle_graph(n)
adjm_cupy = cp.array(nx.to_numpy_array(graph))

start_time = time.time()

eigenvalues, eigenvectors = cp.linalg.eigh(adjm_cupy)
diag = cp.diag(cp.exp(-1j * eigenvalues * time_val)).diagonal()
operatorSpectralCupy = cp.multiply(eigenvectors, diag)
operatorSpectralCupy = cp.matmul(operatorSpectralCupy, eigenvectors.conjugate().transpose())


end_time = time.time()
numpy_time = end_time - start_time
print(f"Expm execution time: {numpy_time} seconds")

Expm execution time: 18.293574333190918 seconds


In [ ]:
# n2 = 5000
# time_val2 = 5
# graph2 = nx.cycle_graph(n2)
# adjm_cupy2 = cp.array(nx.to_numpy_array(graph2))

# def calculateSpecDecomp_cupy(adjm_cupy2, time_val2):
#     eigenvalues, eigenvectors = cp.linalg.eigh(adjm_cupy2)
#     diag = cp.diag(cp.exp(-1j * eigenvalues * time_val2)).diagonal()
#     operatorSpectralCupy = cp.multiply(eigenvectors, diag)
#     operatorSpectralCupy = cp.matmul(operatorSpectralCupy, eigenvectors.conjugate().transpose())
#     return operatorSpectralCupy

# print("Using CuPy benchmarking tool")

# print(benchmark(calculateSpecDecomp_cupy,(adjm_cupy2, time_val2,), n_repeat=1))


############## ------ ################ ------ ######################

# I dont really understand why the CPU and GPU results from this function are the same, 
#but GPU is half of the time in the cells above.

#Wacky results with this method.

Using CuPy benchmarking tool
calculateSpecDecomp_cupy:    CPU: 1637018.200 us     GPU-0: 6481755.371 us


In [35]:
import numpy as np

# Compare the two arrays using np.allclose
are_close = np.allclose(operatorSpectral, operatorSpectralCupy, atol=1e7)

# Print the result of the comparison
print(f"Are the two arrays approximately equal? {are_close}")

Are the two arrays approximately equal? True
